In [1]:
import json
import pandas as pd
import shapely
import geopandas as gpd
from shapely.geometry import LineString, Point
import pandana
import osmnet

In [2]:
od=pd.read_csv(f'data/Gipuzkoa_no_NaNs.csv')

In [4]:
# min_lon, max_lon, min_lat, max_lat = -97.5466, -97.4061, 25.8733, 25.988
min_lon, max_lon, min_lat, max_lat = (
    od['O_long'].min(), od['O_long'].max(),
    od['O_lat'].min(), od['O_lat'].max())

## Drive

In [6]:
# Emplea la libreria osmnet.load para cargar los nodos y edges del network de carreteras

drive_nodes, drive_edges = osmnet.load.network_from_bbox(
    lat_min=min_lat, lng_min=min_lon, lat_max=max_lat, 
    lng_max=max_lon, network_type='drive', two_way=False)

Requesting network data within bounding box from Overpass API in 4 request(s)
Posting to http://www.overpass-api.de/api/interpreter with timeout=180, "{'data': '[out:json][timeout:180];(way["highway"]["highway"!~"cycleway|footway|path|pedestrian|steps|track|proposed|construction|bridleway|abandoned|platform|raceway|service"]["motor_vehicle"!~"no"]["motorcar"!~"no"]["service"!~"parking|parking_aisle|driveway|emergency_access"](42.97819870,-2.54228824,43.18808542,-2.13466048);>;);out;'}"
Downloaded 15,203.7KB from www.overpass-api.de in 1.49 seconds
Posting to http://www.overpass-api.de/api/interpreter with timeout=180, "{'data': '[out:json][timeout:180];(way["highway"]["highway"!~"cycleway|footway|path|pedestrian|steps|track|proposed|construction|bridleway|abandoned|platform|raceway|service"]["motor_vehicle"!~"no"]["motorcar"!~"no"]["service"!~"parking|parking_aisle|driveway|emergency_access"](43.18572840,-2.54228824,43.39254746,-2.13172333);>;);out;'}"
Downloaded 6,851.9KB from www.ove

In [7]:
drive_nodes=drive_nodes.rename(columns = {'x': 'lon', 'y': 'lat'})

In [8]:
# Asigna a cada edge la longitud y latitud de salida y llegada. Edge solo tenia el id de los nodos, y con merge se le añade las caracteristicas.

drive_edges=drive_edges.merge(drive_nodes, how='left', left_on='from', right_index=True).rename(
    columns={'lon': 'from_lon', 'lat': 'from_lat'})
drive_edges=drive_edges.merge(drive_nodes, how='left', left_on='to', right_index=True).rename(
    columns={'lon': 'to_lon', 'lat': 'to_lat'})

In [9]:
# Calcular la geometría (línea) entre 'from' y 'to' de cada fila en df drive_edges con la biblioteca Shapely y almacenando estas geometrías en la nueva columna 'geometry'. Esta técnica es útil cuando estás trabajando con datos geoespaciales y necesitas representar visualmente las líneas entre nodos.

drive_edges['geometry']=drive_edges.apply(
    lambda row: LineString([[row['from_lon'], row['from_lat']],
                             [row['to_lon'], row['to_lat']]]), axis=1)

In [10]:
# Crear un geodataframe

drive_edges_gdf=gpd.GeoDataFrame(data=drive_edges, geometry='geometry', crs='epsg:4326')
drive_edges_gdf

from          to    distance  \
25432201   25439650      25432201    25439650  155.915932   
25439650   25432201      25439650    25432201  155.915932   
           25439602      25439650    25439602   58.853888   
25439602   25439650      25439602    25439650   58.853888   
25439664   25439669      25439664    25439669  103.042800   
...                           ...         ...         ...   
307948700  3898364377   307948700  3898364377  657.049391   
           2424397909   307948700  2424397909   57.454776   
2424397909 307948700   2424397909   307948700   57.454776   
           487091090   2424397909   487091090  200.004057   
487091090  2424397909   487091090  2424397909  200.004057   

                                        name      ref        highway service  \
25432201   25439650    Fermin Calbeton kalea      NaN       tertiary     NaN   
25439650   25432201    Fermin Calbeton kalea      NaN       tertiary     NaN   
           25439602             Plaza Barria      NaN       tertiary     NaN   
25439602   25439650             Plaza Barria      NaN       tertiary     NaN   
25439664   25439669      Zuloagatarren kalea      NaN  living_street     NaN   
...                                      ...      ...            ...     ...   
307948700  3898364377                    NaN  GI-2130      secondary     NaN   
           2424397909                    NaN  GI-2130      secondary     NaN   
2424397909 307948700                     NaN  GI-2130      secondary     NaN   
           487091090                     NaN  GI-2130      secondary     NaN   
487091090  2424397909                    NaN  GI-2130      secondary     NaN   

                      bridge tunnel access  ... area width junction  from_lon  \
25432201   25439650      NaN    NaN    NaN  ...  NaN   NaN      NaN -2.472783   
25439650   25432201      NaN    NaN    NaN  ...  NaN   NaN      NaN -2.471287   
           25439602      NaN    NaN    NaN  ...  NaN   NaN      NaN -2.471287   
25439602   25439650      NaN    NaN    NaN  ...  NaN   NaN      NaN -2.470583   
25439664   25439669      NaN    NaN    NaN  ...  NaN   NaN      NaN -2.470804   
...                      ...    ...    ...  ...  ...   ...      ...       ...   
307948700  3898364377    NaN    NaN    NaN  ...  NaN   NaN      NaN -2.007523   
           2424397909    NaN    NaN    NaN  ...  NaN   NaN      NaN -2.007523   
2424397909 307948700     NaN    NaN    NaN  ...  NaN   NaN      NaN -2.006904   
           487091090     NaN    NaN    NaN  ...  NaN   NaN      NaN -2.006904   
487091090  2424397909    NaN    NaN    NaN  ...  NaN   NaN      NaN -2.004505   

                        from_lat        id_x    to_lon     to_lat        id_y  \
25432201   25439650    43.184141    25432201 -2.471287  43.185020    25439650   
25439650   25432201    43.185020    25439650 -2.472783  43.184141    25432201   
           25439602    43.185020    25439650 -2.470583  43.185149    25439602   
25439602   25439650    43.185149    25439602 -2.471287  43.185020    25439650   
25439664   25439669    43.185877    25439664 -2.470002  43.185159    25439669   
...                          ...         ...       ...        ...         ...   
307948700  3898364377  43.144080   307948700 -2.015493  43.143041  3898364377   
           2424397909  43.144080   307948700 -2.006904  43.143830  2424397909   
2424397909 307948700   43.143830  2424397909 -2.007523  43.144080   307948700   
           487091090   43.143830  2424397909 -2.004505  43.143418   487091090   
487091090  2424397909  43.143418   487091090 -2.006904  43.143830  2424397909   

                                                                geometry  
25432201   25439650    LINESTRING (-2.47278 43.18414, -2.47129 43.18502)  
25439650   25432201    LINESTRING (-2.47129 43.18502, -2.47278 43.18414)  
           25439602    LINESTRING (-2.47129 43.18502, -2.47058 43.18515)  
25439602   25439650    LINESTRING (-2.47058 43.18515, -2.47129 43.18502)  
25439664   254

### Ensure all roads have valid speed

In [11]:
# Eliminar "_link" de los valores de la columna highway

drive_edges_gdf['highway']=drive_edges_gdf['highway'].apply(lambda hw: hw.replace('_link', ''))

# Primero, se llenan los valores nulos con ceros, y luego se convierten los valores restantes en enteros, extrayendo el componente numérico y eliminando cualquier texto adicional (de 60 km/h a 60).

drive_edges_gdf['maxspeed']=drive_edges_gdf['maxspeed'].fillna(0)
drive_edges_gdf['maxspeed']=drive_edges_gdf['maxspeed'].apply(lambda s: int(str(s).split(' ')[0]))

# Agrupa los highways por tipo y calcula y guarda la moda de cada tipo en el elemento "grouped"

grouped = drive_edges_gdf.loc[drive_edges_gdf['maxspeed']>0].groupby('highway')['maxspeed'].agg(pd.Series.mode)
for t in drive_edges_gdf.highway.unique():
    if t not in grouped.keys():
        grouped[t] = grouped["unclassified"]
        
# Replace zero values with group mode
drive_edges_gdf.loc[drive_edges_gdf['maxspeed'] == 0, 'maxspeed'] = drive_edges_gdf.loc[drive_edges_gdf['maxspeed'] == 0, 'highway'].apply(lambda x: grouped[x])

In [12]:
# Add m/s speed and time as a function of distance and speed.

drive_edges_gdf['speed_m_s']=drive_edges_gdf['maxspeed']*1000/3600
drive_edges_gdf['drive_time_s']=drive_edges_gdf['distance']/drive_edges_gdf['speed_m_s']

## Create Shapefile for GAMA

In [13]:
drive_edges_gdf.to_file(f"data/drive_shapefile.shp", driver='ESRI Shapefile')

C:\Users\iazka\AppData\Local\Temp\ipykernel_19508\4092825897.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  drive_edges_gdf.to_file(f"data/drive_shapefile.shp", driver='ESRI Shapefile')


### Create pandana network

In [14]:
drive_net_pdna=pandana.network.Network(
    drive_nodes['lon'], drive_nodes['lat'], 
    drive_edges_gdf['from'], drive_edges_gdf['to'], 
    drive_edges_gdf[['distance','speed_m_s','drive_time_s']], twoway=False)

In [15]:
drive_net_pdna.save_hdf5('networks/drive_net.h5')

In [28]:
drive_nodes.to_csv(f"data/drive_nodes.csv")

In [14]:
# drive_edges_gdf.to_file('../data/drive_net.geojson', index=False)
# drive_edges_gdf.to_file('../front_end/data/drive_net.geojson', index=False)
# drive_nodes.to_csv('../data/drive_nodes.csv')

## Walk and Bike

In [5]:
walk_nodes, walk_edges = osmnet.load.network_from_bbox(
    lat_min=min_lat, lng_min=min_lon, lat_max=max_lat, 
    lng_max=max_lon, network_type='walk', two_way=False)

Requesting network data within bounding box from Overpass API in 4 request(s)
Posting to http://www.overpass-api.de/api/interpreter with timeout=180, "{'data': '[out:json][timeout:180];(way["highway"]["highway"!~"motor|proposed|construction|abandoned|platform|raceway"]["foot"!~"no"]["pedestrians"!~"no"](42.97939051,-2.54228822,43.18868130,-2.13466437);>;);out;'}"
Downloaded 42,786.2KB from www.overpass-api.de in 3.12 seconds
Posting to http://www.overpass-api.de/api/interpreter with timeout=180, "{'data': '[out:json][timeout:180];(way["highway"]["highway"!~"motor|proposed|construction|abandoned|platform|raceway"]["foot"!~"no"]["pedestrians"!~"no"](43.18632431,-2.54228822,43.39254746,-2.13173569);>;);out;'}"
Downloaded 21,365.7KB from www.overpass-api.de in 1.24 seconds
Posting to http://www.overpass-api.de/api/interpreter with timeout=180, "{'data': '[out:json][timeout:180];(way["highway"]["highway"!~"motor|proposed|construction|abandoned|platform|raceway"]["foot"!~"no"]["pedestrians"!

In [6]:
walk_nodes=walk_nodes.rename(columns = {'x': 'lon', 'y': 'lat'})

In [7]:
walk_edges=walk_edges.merge(walk_nodes, how='left', left_on='from', right_index=True).rename(
    columns={'lon': 'from_lon', 'lat': 'from_lat'})
walk_edges=walk_edges.merge(walk_nodes, how='left', left_on='to', right_index=True).rename(
    columns={'lon': 'to_lon', 'lat': 'to_lat'})

In [8]:
walk_edges['geometry']=walk_edges.apply(
    lambda row: LineString([[row['from_lon'], row['from_lat']],
                             [row['to_lon'], row['to_lat']]]), axis=1)

In [26]:
walk_edges_gdf=gpd.GeoDataFrame(data=walk_edges, geometry='geometry', crs='epsg:4326')

In [11]:
walk_nodes.to_csv('data/walk_nodes.csv')

### Create walk pandana network

In [27]:
walk_net_pdna=pandana.network.Network(
    walk_nodes['lon'], walk_nodes['lat'], 
    walk_edges_gdf['from'], walk_edges_gdf['to'], 
    walk_edges_gdf[['distance']], twoway=False)

In [28]:
walk_net_pdna.save_hdf5('networks/walk_net.h5')

In [22]:
# walk_edges_gdf.to_file('../data/walk_net.geojson', index=False)
# walk_edges_gdf.to_file('../front_end/data/walk_net.geojson', index=False)
# walk_nodes.to_csv('../data/walk_nodes.csv')

### Create bike pandana network

In [23]:
walk_net_pdna.save_hdf5('networks/bike_net.h5')

In [24]:
# walk_edges_gdf.to_file('../data/bike_net.geojson', index=False)
# walk_edges_gdf.to_file('../front_end/data/bike_net.geojson', index=False)
# walk_nodes.to_csv('../data/bike_nodes.csv')